In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geoip2.database
import myLogReader as mlr
import re
import os
import sys
import datetime as dt

%matplotlib inline

## Automate - Read and Prep log data into DF

In [2]:
logsPath = '../data/logs'
geoLiteIPDBPath = '../data/GeoLite2-City_20181009/GeoLite2-City.mmdb'

#Create a myLogReader object
myLogReader = mlr.log()
#Open Reader
myLogReader.openReader(geoLiteIPDBPath)

In [3]:
df =  myLogReader.readLogs(logsPath,1)

../data/logs\u_ex171212.log


In [4]:
#Close Reader
myLogReader.closeReader()

In [5]:
df.head()

,date,cs-username,client-ip,time-taken(ms),client-device,client-browser,client-webPage,client-city,client-country,date-IsWeekday,date-calendar-week,date-year,calendar-year-week
0,2017-12-12,u3154795,137.92.194.247,546,Mobile,Safari,Mobile,Belconnen,Australia,1,50,2017,2017-50
1,2017-12-12,u3154795,137.92.194.247,218,Mobile,Safari,Mobile,Belconnen,Australia,1,50,2017,2017-50
2,2017-12-12,u3154795,137.92.194.247,140,Mobile,Safari,Mobile,Belconnen,Australia,1,50,2017,2017-50
3,2017-12-12,u3154795,137.92.194.247,2546,Mobile,Safari,Mobile,Belconnen,Australia,1,50,2017,2017-50
4,2017-12-12,u3154795,137.92.194.247,156,Mobile,Safari,Shared,Belconnen,Australia,1,50,2017,2017-50


#### 2- Automate load logs and aggregate data

In [ ]:
#df.to_csv('test.csv')

In [ ]:
(df.groupby(by=['calendar-year-week'])
            .agg({'client-ip': pd.Series.nunique})
            .rename(columns = {'client-ip':'client-ip-unique-count'}))

(df.groupby(by=['calendar-year-week'])
            .agg({'cs-username': pd.Series.nunique})
            .rename(columns = {'cs-username':'cs-username-unique-count'}))

(df.groupby(by=['calendar-year-week'])
            .agg({'client-ip': pd.Series.count,'time-taken(ms)' : pd.Series.sum})
            .rename(columns = {'client-ip':'client-ip-unique-count','time-taken(ms)' :'time-taken(ms)-sum' }))


(df.groupby(by=['calendar-year-week','client-browser'])
            .agg({'client-browser': pd.Series.count})
            .rename(columns = {'client-browser':'client-browser-count'})      
            .reset_index(level=1)
            .pivot(columns='client-browser',values='client-browser-count')
            .rename(columns = {'Chrome':'Chrome-count'
                              ,'Firefox':'Firefox-count'
                              ,'Other':'Other-count'
                              ,'Safari':'Safari-count'}))



(df.groupby(by=['calendar-year-week','client-device'])
            .agg({'client-device': pd.Series.count})  
            .rename(columns = {'client-device':'client-device-count'})      
            .reset_index(level=1)
            .pivot(columns='client-device',values='client-device-count')
            .rename(columns = {'Desktop':'Desktop-count'
                              ,'Mobile':'Mobile-count'}))

In [ ]:

(df.groupby(by=['calendar-year-week','client-browser'])
            .agg({'client-browser': pd.Series.count})
            .rename(columns = {'client-browser':'client-browser-count'})      
            .reset_index(level=1)
            .pivot(columns='client-browser',values='client-browser-count')
)
#df.head()

In [ ]:
df.info()

In [ ]:
df['client-ip'].describe()

In [ ]:
df.loc[:, df.dtypes == object].describe()

In [ ]:
df.isna().sum()